# BD Processing

In [ ]:
#- CONFIGURACIÓN Y PRE-PROCESAMIENTO 

In [2]:
# Localizar y configurar ruta Spark para PySpark

import findspark
findspark.init() 

In [3]:
# Importar Módulo principal de PySpark y Librerías

import pyspark
import pandas as pd
from pyspark.sql.window import Window

In [4]:
# Crear SparkSession y SparkContext 

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('Practice_BDP')\
        .config("spark.sql.repl.eagerEval.enabled", True)\
        .getOrCreate()

In [5]:
# Importar Funciones y Clases 

from pyspark.sql.functions import *
from pyspark.sql.types import *

In [6]:
# Importar datos (Dataset) con PySpark

df1_whreport = spark.read.csv("C:/Users/e-bus/OneDrive/Desktop/Práctica_BD_Processing/Datasets/world-happiness-report.csv", sep=",", header=True, inferSchema=True)
df2_whreport_21 = spark.read.csv("C:/Users/e-bus/OneDrive/Desktop/Práctica_BD_Processing/Datasets/world-happiness-report-2021.csv", sep=",", header=True, inferSchema=True)

In [515]:
# Validar datos 
# 2005-2020

display(df1_whreport.limit(10))

Country name,Year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
Afghanistan,2008,3.724,7.37,0.451,50.8,0.718,0.168,0.882,0.518,0.258
Afghanistan,2009,4.402,7.54,0.552,51.2,0.679,0.19,0.85,0.584,0.237
Afghanistan,2010,4.758,7.647,0.539,51.6,0.6,0.121,0.707,0.618,0.275
Afghanistan,2011,3.832,7.62,0.521,51.92,0.496,0.162,0.731,0.611,0.267
Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.71,0.268
Afghanistan,2013,3.572,7.725,0.484,52.56,0.578,0.061,0.823,0.621,0.273
Afghanistan,2014,3.131,7.718,0.526,52.88,0.509,0.104,0.871,0.532,0.375
Afghanistan,2015,3.983,7.702,0.529,53.2,0.389,0.08,0.881,0.554,0.339
Afghanistan,2016,4.22,7.697,0.559,53.0,0.523,0.042,0.793,0.565,0.348
Afghanistan,2017,2.662,7.697,0.491,52.8,0.427,-0.121,0.954,0.496,0.371


In [450]:
# Validar datos 
# 2021
display(df2_whreport_21.limit(10))

Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
Finland,Western Europe,7.842,0.032,7.904,7.78,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
Denmark,Western Europe,7.62,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.03,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
Switzerland,Western Europe,7.571,0.036,7.643,7.5,11.117,0.942,74.4,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
Iceland,Western Europe,7.554,0.059,7.67,7.438,10.878,0.983,73.0,0.955,0.16,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.17,2.967
Netherlands,Western Europe,7.464,0.027,7.518,7.41,10.932,0.942,72.4,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798
Norway,Western Europe,7.392,0.035,7.462,7.323,11.053,0.954,73.3,0.96,0.093,0.27,2.43,1.543,1.108,0.782,0.703,0.249,0.427,2.58
Sweden,Western Europe,7.363,0.036,7.433,7.293,10.867,0.934,72.7,0.945,0.086,0.237,2.43,1.478,1.062,0.763,0.685,0.244,0.448,2.683
Luxembourg,Western Europe,7.324,0.037,7.396,7.252,11.647,0.908,72.6,0.907,-0.034,0.386,2.43,1.751,1.003,0.76,0.639,0.166,0.353,2.653
New Zealand,North America and...,7.277,0.04,7.355,7.198,10.643,0.948,73.4,0.929,0.134,0.242,2.43,1.4,1.094,0.785,0.665,0.276,0.445,2.612
Austria,Western Europe,7.268,0.036,7.337,7.198,10.906,0.934,73.3,0.908,0.042,0.481,2.43,1.492,1.062,0.782,0.64,0.215,0.292,2.784


In [460]:
# Validar y limpiar Datos (Missings Data) 
# 2005-2020
# Life Ladder

df1_whreport.filter(df1_whreport["Life Ladder"].isNull()).limit(10)

Country name,Year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect


In [432]:
# Validar y limpiar Datos (Missings Data)
# 2021
# Ladder score

df2_whreport_21.filter(df2_whreport_21["Ladder score"].isNull()).limit(10)

Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual


In [ ]:
#- PROCESAMIENTO Y VISUALIZACIÓN
# Respuestas 

In [433]:
# 1. ¿Cuál es el país más “feliz” del 2021 según la data? 
# Validar los Países por: “Life Ladder” - "Ladder score"

df2_whreport_21.orderBy(desc("Ladder score")).limit(1)

Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
Finland,Western Europe,7.842,0.032,7.904,7.78,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253


In [9]:
# 2. ¿Cuál es el país más “feliz” del 2021 por continente según la data?
# Agrupar los datos por Continente (Regional indicator)
# Validar índice de felicidad por País  (Ladder score as ls)

ls = df2_whreport_21.select("Country name", "Regional indicator", "Ladder score")
ls = df2_whreport_21.groupBy("Regional indicator").agg(max("Ladder score").alias("max_ls"), first("Country name").alias("Country")).sort(desc("max_ls"))
ls.limit(10)

Regional indicator,max_ls,Country
Western Europe,7.842,Finland
North America and...,7.277,New Zealand
Middle East and N...,7.157,Israel
Latin America and...,7.069,Costa Rica
Central and Easte...,6.965,Czech Republic
East Asia,6.584,Taiwan Province o...
Southeast Asia,6.377,Singapore
Commonwealth of I...,6.179,Uzbekistan
Sub-Saharan Africa,6.049,Mauritius
South Asia,5.269,Nepal


In [8]:
# 3. ¿Cuál es el país que más veces ocupó el primer lugar en todos los años?
# Seleccionar Columnas de referencia para el Ranking (País, Año, Life Ladder)
# Realizar conteo de ocupación del primer lugar (por País) 
# Validar el País que ha estado más veces en el Primer Lugar

df1_whreport = df1_whreport.select("Country name", "Year", "Life Ladder")
df1_whreport = df1_whreport.orderBy(["Country name", "Year"]).withColumn("rank", dense_rank().over(Window.partitionBy("Year").orderBy(desc("Life Ladder"))))
df1_whreport = df1_whreport.filter(df1_whreport.rank == 1).groupBy("Country name").agg(count("*").alias("Ranked_first"))
df1_whreport = df1_whreport.orderBy(desc("Ranked_first")).limit(10)
df1_whreport.limit(5)


Country name,Ranked_first
Denmark,7
Finland,6
Norway,1
Switzerland,1
Canada,1


In [516]:
# 4. ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?
# Seleccionar columnas para filtrado
# Filtrar los datos (Año=2020 & Ordenar GDP)
# Validar el primer registro (País con mayor GDP año=2020)
# Filtrar datos para obtener (Puesto de Felicidad País con mayor GDP)
# Validar el primer registro del DataFrame filtrado
# Validar la posición del País (Mayor GDP)

df1_whreport = df1_whreport.select("Country name", "Year", "Log GDP per capita", "Life Ladder")
df1_whreport_filter = df1_whreport.filter(df1_whreport["Year"] == 2020).orderBy(desc("Log GDP per capita"))
pais_mayor_gdp = df1_whreport_filter.first()["Country name"]
df1_whreport_filter2 = df1_whreport.filter(df1_whreport["Country name"] == pais_mayor_gdp)
data_pais_mayor_gdp = df1_whreport_filter2.first()
posición_país = data_pais_mayor_gdp["Life Ladder"]
df1_whreport_filter.limit(3)

Country name,Year,Log GDP per capita,Life Ladder
Ireland,2020,11.323,7.035
Switzerland,2020,11.081,7.508
United Arab Emirates,2020,11.053,6.458


In [538]:
# 5. ¿En que porcentaje a variado a nivel mundial el GDP promedio del 2020 respecto al 2021? ¿Aumentó o disminuyó?
# Seleccionar GDP (2020 & 2021)
# Calcular el promedio de GDP por año
# Calcular el comportamiento porcentual
# Determinar los cambios en el comportamiento del GDP

gdp_20 = df1_whreport.filter(df1_whreport.Year == 2020).agg(avg("Log GDP per capita")).collect()[0][0]
gdp_21 = df2_whreport_21.select(avg("Logged GDP per capita")).collect()[0][0]
display(gdp_20)
display(gdp_21)

perc_var = ((gdp_21 - gdp_20) / gdp_20) * 100
print(f"La variación porcentual del GDP promedio de 2020 a 2021 es de {perc_var:.2f}%.")


9.751329545454546

9.432208053691273

La variación porcentual del GDP promedio de 2020 a 2021 es de -3.27%.


In [548]:
# 6. ¿Cuál es el país con mayor expectativa de vida (“Healthy life expectancy at birth”)? Y ¿Cuánto tenia en ese indicador en el 2019?
# Validar datos de expectativa de vida por país y año (En los dos archivos)
# Determinar el país que cumple la condición (ordenar en orden desc)
# Validar datos (Año=2019)

#Periodo 2005-2020
df1_whreport_filter = df1_whreport.select("Country name", "Year", "Healthy life expectancy at birth") \
    .where(df1_whreport.Year == df1_whreport.select("Year").distinct().orderBy(desc("Year")).first()[0])
df1_whreport_sort = df1_whreport_filter.orderBy(desc("Healthy life expectancy at birth"))
country_Hhle = df1_whreport_sort.select("Country name", "Healthy life expectancy at birth").first()
print(f"El país con la mayor expectativa de vida periodo 2005 - 2020 {country_Hhle['Country name']} con un valor de {country_Hhle['Healthy life expectancy at birth']:.2f}.")

#Año 2021
df2_whreport_21_filter = df2_whreport_21.select("Country name", "Healthy life expectancy")
df2_whreport_21_sort = df2_whreport_21_filter.orderBy(desc("Healthy life expectancy"))
country_Hhle_21 = df2_whreport_21_sort.select("Country name", "Healthy life expectancy").first()
print(f"El país con la mayor expectativa de vida en el 2021 es {country_Hhle_21['Country name']} con un valor de {country_Hhle_21['Healthy life expectancy']:.2f}.")

#Año 2019
df1_whreport_filter_19 = df1_whreport.select("Country name", "Healthy life expectancy at birth", "year") \
                              .filter(col("year") == 2019)
country_Hhle_19 = df1_whreport_filter_19.orderBy(desc("Healthy life expectancy at birth")).first()
print(f"El país con mayor expectativa de vida en 2019 es {country_Hhle_19['Country name']} con {country_Hhle_19['Healthy life expectancy at birth']} años.")


El país con la mayor expectativa de vida periodo 2005 - 2020 Japan con un valor de 75.20.
El país con la mayor expectativa de vida en el 2021 es Singapore con un valor de 76.95.
El país con mayor expectativa de vida en 2019 es Singapore con 77.1 años.
